In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install accelerate -U

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

# Germeval 2024: LLM Classification

In [91]:
import sys
import urllib
import datasets
from datasets import load_dataset
from datasets import Dataset
from datasets import DatasetDict
import matplotlib.pyplot as plt
import torch
import transformers
import numpy as np
import pandas as pd

datapath = "./data/"

In [92]:
train = pd.read_csv(datapath + "train.csv")
trial = pd.read_csv(datapath + "trial.csv")
augmented = pd.read_csv(datapath + "augmented.csv")

train = pd.concat([train, trial,augmented], join="inner")

print(train.shape)

train = train.drop_duplicates(subset=['sent-id'], keep='last')
train = train[train["num_statements"] > 0]
print(train.shape)
test = pd.read_csv(datapath + "test.csv")
test = test[test["num_statements"] > 0]
#intersection = train.columns.intersection(test.columns)
intersection = ["num_statements", "phrase"]
train = train[intersection]
print(train.shape)
test = test[intersection]
train.shape, test.shape


(4177, 10)
(3725, 10)
(3725, 2)


((3725, 2), (416, 2))

In [93]:
train = train.rename(columns= {"num_statements": "label"})
test = test.rename(columns= {"num_statements": "label"})


In [152]:
def label_names(x):
  if x==0:
    return "zero"
  elif x == 1:
    return "one"
  elif x==2:
    return "two"
  elif x==3:
    return "three"
  elif x==4:
    return "four"
  elif x==5:
    return "five"
  elif x==6:
    return "six"
  elif x==7:
    return "seven"
  elif x==8:
    return "eight"
  elif x==9:
    return "nine or more"

In [153]:
label_names(4)

'four'

In [95]:
train["label"] = train["label"].apply(label_names)
test["label"] = test["label"].apply(label_names)


In [96]:
train

,label,phrase
0,one,Eine sehr bekannte Alchemisten war Maria die J...
1,one,Alchemisten wollen etwas ver-bessern.
2,two,"Zum Beispiel, \newline wollen sie kranke Mensc..."
3,one,Alchemisten haben Athanoren benutzt.
4,one,Sue haben Schwarz-pulver erfunden.
...,...,...
1202,three,"Sie treffen sich mit \newline, um gemeinsam ka..."
1203,two,Im Jahr 1877 heirateten Alexander Graham Bell ...
1204,two,Der Abfall landet im Mülleimer oder in Tüten.
1205,two,Die Berlin-Blockade dauerte vom 23. Juni 1948 ...


In [97]:
train_ds = Dataset.from_pandas(train, split="train")
test_ds = Dataset.from_pandas(test, split="test")
ds = DatasetDict({"train": train_ds,"test": test_ds})

In [98]:
ds = ds.class_encode_column("label")

Casting to class labels:   0%|          | 0/3725 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/416 [00:00<?, ? examples/s]

In [99]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'phrase', '__index_level_0__'],
        num_rows: 3725
    })
    test: Dataset({
        features: ['label', 'phrase'],
        num_rows: 416
    })
})

In [100]:
ds["train"].features

{'label': ClassLabel(names=['five', 'four', 'one', 'six', 'three', 'two'], id=None),
 'phrase': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [154]:
from collections import Counter
Counter(ds["train"]["label"]), Counter(ds["test"]["label"])

(Counter({2: 1545, 5: 1478, 1: 114, 4: 576, 0: 9, 3: 3}),
 Counter({2: 258, 4: 123, 1: 6, 3: 28, 0: 1}))

In [101]:
from transformers import AutoTokenizer

checkpoint = "bert-base-german-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [102]:
def tokenize(batch):
    return tokenizer(batch["phrase"], padding=True, truncation=True)

In [103]:
ds_tokenized = ds.map(tokenize, batched=True, batch_size=1000)


Map:   0%|          | 0/3725 [00:00<?, ? examples/s]

Map:   0%|          | 0/416 [00:00<?, ? examples/s]

In [104]:
# HuggingFace account necessary
from huggingface_hub import notebook_login

hugging = True

if hugging == True:
    notebook_login()

# if working from a terminal use $ huggingface-cli login

In [105]:
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = len(ds["train"].features["label"].names)

model = (AutoModelForSequenceClassification
         .from_pretrained(checkpoint, num_labels=num_labels)
         .to(device))

In [106]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [107]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(ds_tokenized["train"]) // batch_size
model_path = f"{checkpoint}-germeval2024"
training_args = TrainingArguments(output_dir=model_path,
                                  num_train_epochs=5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy="steps",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=hugging, # HuggingFace account necessary
                                  log_level="error")

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=ds_tokenized["train"],
                  eval_dataset=ds_tokenized["test"],
                  tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [108]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
58,0.917400,1.527105,0.526442,0.549939
116,0.502200,1.816397,0.564904,0.572039
174,0.241100,2.156423,0.555288,0.567818
232,0.107800,2.618912,0.526442,0.558222
290,0.045100,2.833479,0.514423,0.547300


TrainOutput(global_step=295, training_loss=0.35753469649007763, metrics={'train_runtime': 155.8311, 'train_samples_per_second': 119.52, 'train_steps_per_second': 1.893, 'total_flos': 507290680336500.0, 'train_loss': 0.35753469649007763, 'epoch': 5.0})

In [124]:
preds_output = trainer.predict(ds_tokenized["test"])

In [125]:
preds_output.metrics

{'test_loss': 2.8278965950012207,
 'test_accuracy': 0.5168269230769231,
 'test_f1': 0.5487515992323684,
 'test_runtime': 0.7686,
 'test_samples_per_second': 541.212,
 'test_steps_per_second': 9.107}

In [166]:
y_preds = np.argmax(preds_output.predictions, axis=1)
y_test = np.array(ds_tokenized["test"]["label"])

In [167]:
from collections import Counter
Counter(y_preds), Counter(y_test)

(Counter({2: 236, 5: 155, 4: 25}),
 Counter({2: 258, 4: 123, 1: 6, 3: 28, 0: 1}))

In [168]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_test,y_preds, zero_division = True, labels = label_names(labels)))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00         1
           1       1.00      0.00      0.00         6
           2       0.89      0.81      0.85       258
           3       1.00      0.00      0.00        28
           4       0.24      0.05      0.08       123
           5       0.00      1.00      0.00         0

    accuracy                           0.52       416
   macro avg       0.69      0.31      0.15       416
weighted avg       0.70      0.52      0.55       416



In [169]:
print(confusion_matrix(y_test,y_preds))

[[  0   0   0   0   0   1]
 [  0   0   0   0   3   3]
 [  0   0 209   0   2  47]
 [  0   0   0   0  14  14]
 [  0   0  27   0   6  90]
 [  0   0   0   0   0   0]]


In [170]:
ds["train"].features["label"].names, ds["test"].features

(['five', 'four', 'one', 'six', 'three', 'two'],
 {'label': ClassLabel(names=['five', 'four', 'one', 'three', 'two'], id=None),
  'phrase': Value(dtype='string', id=None)})

In [46]:
from transformers import pipeline

# Change `Wiebke` to your Hub username
model_id = "Wiebke/german-base-cased-germeval2024-stage"
classifier = pipeline("text-classification", model=model_id)

OSError: Wiebke/german-base-cased-germeval2024-stage is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
text = ["Dieser Satz enthält mit großer Sicherheit mehr als zwei Statements, obwohl kein Punkt vorkommt."]
test_class = classifier(text, return_all_scores=True, function_to_apply="softmax")
test_class